In [1]:
using CSV
using DataFrames
using Dates

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/output_HHS_US_2021-02-13_one_column.csv"))

,Column1,hospital_name,ccn,collection_week,fips_code,hospitalization_percentage,ICU_percentages,hospitalization_2021-01-19,target_date
,Int64,String,String?,Date,Float64,Float64?,Float64?,Float64?,Date
1,10,PRATTVILLE BAPTIST HOSPITAL,010108,2021-01-08,1001.0,1.0,0.130584,5.39817,2021-01-19
2,11,THOMAS HOSPITAL,010100,2021-01-08,1003.0,0.516628,0.265781,7.5387,2021-01-19
3,12,NORTH BALDWIN INFIRMARY,010129,2021-01-08,1003.0,0.165797,0.135714,2.41933,2021-01-19
4,13,SOUTH BALDWIN REGIONAL MEDICAL CENTER,010083,2021-01-08,1003.0,0.317575,0.292683,4.6341,2021-01-19
5,14,MEDICAL CENTER BARBOUR,010069,2021-01-08,1005.0,1.0,0.277778,1.34824,2021-01-19
6,15,BIBB MEDICAL CENTER,010058,2021-01-08,1007.0,0.791667,missing,1.33872,2021-01-19
7,16,ST VINCENTS BLOUNT,011305,2021-01-08,1009.0,0.958333,0.102041,3.82677,2021-01-19
8,17,BULLOCK COUNTY HOSPITAL,010110,2021-01-08,1011.0,0.0,missing,0.0,2021-01-19
9,18,REGIONAL MEDICAL CENTER OF CENTRAL ALABAMA,010150,2021-01-08,1013.0,0.958333,0.262295,1.08509,2021-01-19


In [4]:
function firstval(xs)
    isbad(x) = isnothing(x) || ismissing(x)
    if all(isbad.(xs))
        return missing
    else
        return xs[findfirst(x -> !isbad(x), xs)]
    end
end;

In [5]:
ccn_lookup_df = combine(groupby(rawdata, [:Column1, :hospital_name]), :ccn => firstval => :ccn)
ccn_lookup = Dict(row.Column1 => row.ccn for row in eachrow(ccn_lookup_df));

In [6]:
data = select(rawdata,
    :hospital_name => :hospital,
    :Column1 => ByRow(x -> ccn_lookup[x]) => :hospital_id,
    :target_date => :date,
    ["hospitalization_2021-01-19", "ICU_percentages"] => ByRow((x,y) -> x*coalesce(y,0.3)) => :admitted_moderate_icu,
    ["hospitalization_2021-01-19", "ICU_percentages"] => ByRow((x,y) -> x*(1.0-coalesce(y,0.3))) => :admitted_moderate_acute,
    "hospitalization_2021-01-19" => :admitted_moderate_allbeds,
)
sort!(data, [:hospital, :hospital_id, :date])

,hospital,hospital_id,date,admitted_moderate_icu,admitted_moderate_acute,admitted_moderate_allbeds
,String,String?,Date,Float64?,Float64?,Float64?
1,ABBEVILLE AREA MEDICAL CENTER,421301,2021-01-19,0.0,0.898606,0.898606
2,ABBEVILLE AREA MEDICAL CENTER,421301,2021-01-20,0.0,0.899638,0.899638
3,ABBEVILLE AREA MEDICAL CENTER,421301,2021-01-21,0.0,0.972093,0.972093
4,ABBEVILLE AREA MEDICAL CENTER,421301,2021-01-22,0.0,0.955333,0.955333
5,ABBEVILLE AREA MEDICAL CENTER,421301,2021-01-23,0.0,0.972609,0.972609
6,ABBEVILLE AREA MEDICAL CENTER,421301,2021-01-24,0.0,1.06283,1.06283
7,ABBEVILLE AREA MEDICAL CENTER,421301,2021-01-25,0.0,1.07168,1.07168
8,ABBEVILLE AREA MEDICAL CENTER,421301,2021-01-26,0.0,1.03571,1.03571
9,ABBEVILLE AREA MEDICAL CENTER,421301,2021-01-27,0.0,1.03516,1.03516


In [7]:
data |> CSV.write("../data/hhs_forecast_2021_01_19.csv")

"../data/hhs_forecast_2021_01_19.csv"